# 1. Importing data and helper functions <a class="anchor" id="importing-data"></a>

In [2]:
import pandas as pd

# Load the first sheet into a DataFrame
df_wb = pd.read_excel("dataset.xlsx", sheet_name=0)  # WELLBEING

# Load the second sheet into another DataFrame
df_mr = pd.read_excel("dataset.xlsx", sheet_name=1)  # MINERALS

In [3]:
df_wb.head()

,STRUCTURE,STRUCTURE_ID,STRUCTURE_NAME,ACTION,REF_AREA,Reference area,MEASURE,Measure,UNIT_MEASURE,Unit of measure,...,OBS_VALUE,Observation value,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals,BASE_PER,Base period
0,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1),Current well-being,I,PRT,Portugal,10_1,Homicides,DT_10P5HB,Deaths per 100 000 inhabitants,...,0.6,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
1,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1),Current well-being,I,PRT,Portugal,10_1,Homicides,DT_10P5HB,Deaths per 100 000 inhabitants,...,0.8,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
2,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1),Current well-being,I,PRT,Portugal,10_1,Homicides,DT_10P5HB,Deaths per 100 000 inhabitants,...,0.5,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
3,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1),Current well-being,I,PRT,Portugal,10_1,Homicides,DT_10P5HB,Deaths per 100 000 inhabitants,...,0.8,NaN,A,Normal value,0,Units,2,Two,NaN,NaN
4,DATAFLOW,OECD.WISE.WDP:DSD_HSL@DF_HSL_CWB(1.1),Current well-being,I,PRT,Portugal,10_1,Homicides,DT_10P5HB,Deaths per 100 000 inhabitants,...,0.6,NaN,A,Normal value,0,Units,2,Two,NaN,NaN


In [6]:
df_mr.head()

,Country,Category,Flow name,Flow code,Flow unit,1970,1971,1972,1973,1974,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,Biomass,Domestic Extraction,DE,t,29427854.0,28777035.0,27200896.0,29147772.0,30519158.0,...,37406375.0,37961488.0,35542253.0,33625557.0,39501360.0,41190154.0,37576686.0,37891437.0,38353304.0,38628527.0
1,Afghanistan,Biomass,Domestic Material Consumption,DMC,t,29476560.0,29136965.0,27290656.0,29112673.0,30433539.0,...,40423146.0,41655757.0,40992559.0,38552083.0,43433438.0,45294467.0,39730808.0,40083922.0,40596386.0,40905900.0
2,Afghanistan,Biomass,Domestic Material Input,DMI,t,29617873.0,29282898.0,27485608.0,29306247.0,30656605.0,...,40740301.0,42111457.0,41553874.0,39515927.0,44428717.0,46059807.0,40992155.0,41370148.0,41905797.0,42238460.0
3,Afghanistan,Biomass,Exports,EXP,t,141313.0,145933.0,194952.0,193574.0,223066.0,...,317155.0,455700.0,561315.0,963844.0,995279.0,765341.0,1261347.0,1286226.0,1309411.0,1332559.0
4,Afghanistan,Biomass,Imports,IMP,t,190019.0,505863.0,284712.0,158475.0,137447.0,...,3333926.0,4149969.0,6011621.0,5890371.0,4927357.0,4869653.0,3415470.0,3478710.0,3552492.0,3609933.0


In [7]:
# clearing rows from a dataset matching a given parameter
def remove_rows_from_dataset(rows, df):
    return df[~df["Country"].isin(rows)]

# used to print the measurement unit
def find_unit(value):
    conversion = {
        "kt" : 1_000,
        "Mt" : 1_000_000,
        "Gt" : 1_000_000_000
    }
    
    for unit, threshold in reversed(conversion.items()):
        if value >= threshold:
            return unit
    
    return "t"

# function to plot a sub-dataset based on a few parameters
def plot_flow_category(data, flow, category, years, ylabel = False, debug = False):
    years_filter = ["Country"] + [str(y) for y in years] 

    data_filters = (data["Flow name"] == flow) & (data["Category"] == category)

    # Filter for this chart only
    data = data[data_filters][years_filter]

    data = data.dropna()

    maximum_year = str(max(years))
    data = data[data[maximum_year] > 0]

    # Sort descending
    data = data.sort_values(maximum_year, ascending=False)

    data = data.head()

    if debug:
        print(data)
        return    

    melted_df = pd.melt(
        data,
        id_vars=["Country"],
        value_vars=years_filter,
        var_name="Year",
        value_name="Value"
    )

    plot = sb.barplot(
        data=melted_df,
        x="Country",
        y="Value",
        hue="Year",
        palette="viridis"
    )

    plt.xticks(data["Country"], rotation=45)
    plot.yaxis.offsetText.set_visible(False)

    plt.xlabel("Country")
    
    if ylabel:
        plt.ylabel(f"{ylabel}")
    else:
        max_value = max(
            value for col in data.columns for value in data[col] 
            if isinstance(value, (int, float))
        )
        
        plt.ylabel(f"{flow} ({find_unit(max_value)})")  # Use the flow unit from the data

    plt.title(f"{flow} by Country for Selected Years")
    plt.legend(title="Year")

    plt.show()

# 2. Cleaning data <a class="anchor" id="cleaning-data"></a>

In [8]:
# removing rows related to continents and organisations, need only countries
rows_to_remove = ["World", "Asia + Pacific", "North America", "EECCA", "West Asia", "Africa", "Europe", "Latin America + Caribbean"]
df_mr = remove_rows_from_dataset(rows_to_remove, df_mr)

### 2.1 Removing unused columns

In [13]:
# is_nan_df = df_mr.isna()

# # If a value is NaN is has True in is_nan_df. We could check if there is any True value
# any(is_nan_df["1970"])

# nan_list = []
# for c in is_nan_df.columns:
#     if any(is_nan_df[c]):
#         nan_list.append(c)

# print(nan_list)
